In [ ]:
import pandas as pd
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
image_folder = os.path.join(parent_dir, "Cuneiform Tests", "imgset")

# list of all images in the image folder and isolate char from iamge name
images = os.listdir(image_folder)
# remove any non-image files
images = [i for i in images if i.endswith(".png")]
char = [""] * len(images)
for i in range(len(images)):
    char[i] = images[i].split(".")[0].split("_")[1]

In [ ]:
# create a dataframe with two columns: imagename and char
df = pd.DataFrame({'image': images, 'character': char})

# save the dataframe to a csv file
df.to_csv('imgset.csv', index=False)

In [ ]:
df.head()

In [ ]:
# drop characters with less than 10 images
df = df.groupby('character').filter(lambda x: len(x) >= 10)

classes = len(df['character'].unique())
print("Number of classes: ", classes, "in ", len(df), " images. Ratio: ", len(df)/classes, " images per class.")

In [ ]:
# use a label encoder to convert the characters to integers
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['character'])

df.head()

In [ ]:
# use StratifiedShuffleSplit to split the data into training and testing sets

from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for train_index, test_index in sss.split(df['image'], df['character']):
    train = df.iloc[train_index]
    test = df.iloc[test_index]

# create a folder for the training and testing images
os.makedirs('imgset/Training_Data', exist_ok=True)
os.makedirs('imgset/Testing_Data', exist_ok=True)

# create df for training and testing images
train.to_csv('imgset/Training_Data/train.csv', index=False)
print("Training data: ", len(train))
test.to_csv('imgset/Testing_Data/test.csv', index=False)
print("Testing data: ", len(test))


In [ ]:
# get pixel width and height of images
from PIL import Image

img = Image.open(os.path.join(image_folder, images[0]))
width, height = img.size
print("Image size: ", width, "x", height)

In [ ]:
# create CNN model using Keras
# each image is 250x250 pixels and grayscale

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.calibration import LabelEncoder

# Create the CNN model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the feature maps
model.add(Flatten())

# Add fully connected layers
# model.add(Dense(256, activation='relu'))


model.add(Dense(3500, activation='relu'))
model.add(Dense(325, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'recall', 'precision'])

# Print the model summary
model.summary()

In [ ]:
# create a generator to load images in batches
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train,
    directory=image_folder,
    x_col='image',
    y_col='character',
    target_size=(50, 50),
    color_mode='grayscale',
    batch_size=5,
    class_mode='categorical')

test_generator = datagen.flow_from_dataframe(
    dataframe=test,
    directory=image_folder,
    x_col='image',
    y_col='character',
    target_size=(50, 50),
    color_mode='grayscale',
    batch_size=5,
    class_mode='categorical')

In [ ]:
# Layer Overview:
# Conv2D: 32 filters, 3x3 kernel, relu activation, input shape 50x50x1
    # used for feature extraction, which means that it extracts the most important features from the input image

# MaxPooling2D: 2x2 pool size
    # for each 2x2 pixel region, the maximum pixel value is taken: used to reduce the spatial dimensions of the output volume

# Flatten: flatten the feature maps
    # used to convert the 3D feature maps to 1D feature vectors: used to connect the convolutional layers to the dense layers

# Dense: 2 neurons, relu activation
    # used for classification: used to connect the flattened feature vectors to the output layer

# Dense: 325 neurons, softmax activation
    # used for classification: used to output the class probabilities


In [ ]:
# train the model, save after each epoch
from keras.callbacks import ModelCheckpoint
import datetime

timestamp = datetime.datetime.now().strftime("%d.%m.%Y")

checkpoint = ModelCheckpoint("model_outputs/callbacks/" + timestamp + ".keras", save_best_only=True)
history = model.fit(train_generator, validation_data=test_generator, epochs=10, callbacks=[checkpoint], verbose=1)

# save model history as csv
history_df = pd.DataFrame(history.history)
history_df.to_csv('history_' + timestamp + '.csv', index=False)

In [ ]:
# create dataframe from csv, using first row as header
history_df = pd.read_csv('model_outputs/cuneiform_model_history.csv')

# plot the training and validation accuracy
import matplotlib.pyplot as plt

plt.plot(history_df['accuracy'])
plt.plot(history_df['val_accuracy'])
# add title and labels
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# save the plot in folder model_outputs
plt.savefig('model_outputs/cuneiform_model_history.png')


In [ ]:
# test the model
#model.evaluate(test_generator)
model.evaluate(train_generator)

# save the model
#model.save('model_outputs/cuneiform_model' + date_time + '.keras')

In [ ]:
# test model with image from folder imgset

import numpy as np
# predict the character for the first 100 images in folder

correct_chars = []

# randomly pick 500 images
import random
random.seed(0)
random.shuffle(images)


for i in range(500):
    img = Image.open(os.path.join(image_folder, images[i]))
    img = img.resize((250, 250))
    img = img.convert('L') # to grayscale
    img = np.array(img)
    img = img.reshape(1, 250, 250, 1)
    prediction = model.predict(img)
    real_char = images[i].split(".")[0].split("_")[1]
    print('\nreal character: ', real_char, '\n' ,'prediction: ' ,le.inverse_transform(np.argmax(prediction, axis=1))[0], '\n')
    if real_char == le.inverse_transform(np.argmax(prediction, axis=1))[0]:
        correct_chars.append(images[i])
    
print("Accuracy: ", len(correct_chars)/500)

In [ ]:
print(correct_chars)